In [2]:
from rdflib import Graph, URIRef, Namespace, Literal, XSD, RDF
from rdflib.plugins.stores.sparqlstore import SPARQLStore
from tqdm.autonotebook import tqdm
import pandas as pd
from copy import deepcopy
import numpy
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict
from rapidfuzz import process, fuzz, distance
import math

#import graph_similarity

import random

import torch
from transformers import BertTokenizer
from transformers import BertModel
from sklearn.metrics.pairwise import cosine_similarity

import concurrent.futures

from rdflib import URIRef, Literal
import string
import dateutil.parser as dparser
import datetime
from rapidfuzz import fuzz, distance
import math
import re

In [159]:
GRAPH_1 = "/Users/yamamotobikutorueiichi/codes/custom_paris/mcu.xml"
GRAPH_1_INV_FUNC_PATH = '/Users/yamamotobikutorueiichi/codes/custom_paris/mcu-inversability.csv'

GRAPH_2 = "/Users/yamamotobikutorueiichi/codes/custom_paris/marvel.xml"
GRAPH_2_INV_FUNC_PATH = '/Users/yamamotobikutorueiichi/codes/custom_paris/marvel-inversability.csv'

#GRAPH_2 = "http://localhost:9999/bigdata/sparql"
SUBJECT = 'subject'
PREDICATE = 'predicate'
INVERSE_FUNCTIONALITY = 'inverse_functionality'
INVERSABILITY = 'inverse_functionality'
FUNCTIONALITY = 'functionality'
MAX_LENGTH_FULL_MATCH = 10000
MAX_SAVED_PREDICATE_COUNTER = 10000000
LABEL_PREDICATE = "http://www.w3.org/2000/01/rdf-schema#label"
FILE_FOLDER = "/Users/yamamotobikutorueiichi/codes/custom_paris/both func and func inv method/sww-swg/"
GRAPH_1_ABSTRACT = "http://dbkwik.webdatacommons.org/ontology/abstract"
GRAPH_2_ABSTRACT = "http://dbkwik.webdatacommons.org/ontology/abstract"

In [4]:
graph_1_inv_func_df = pd.read_csv(GRAPH_1_INV_FUNC_PATH)

#graph_1_predicate_list = threshold_graph_1_inv_func_df[PREDICATE].tolist()

graph_1 = Graph()
graph_1.parse(GRAPH_1)

<Graph identifier=N4a9b981230d34191b0fb93af436ffdb6 (<class 'rdflib.graph.Graph'>)>

In [86]:
for p, o in graph_1.predicate_objects(URIRef('http://dbkwik.webdatacommons.org/marvelcinematicuniverse.wikia.com/resource/Category:TV_Series')):
    print(p)
    print(o)
    print('_____')

http://www.w3.org/2004/02/skos/core#prefLabel
TV Series
_____
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/2004/02/skos/core#Concept
_____
http://www.w3.org/2004/02/skos/core#broader
http://dbkwik.webdatacommons.org/marvelcinematicuniverse.wikia.com/resource/Category:Content
_____
http://www.w3.org/2000/01/rdf-schema#label
TV Series
_____


In [162]:
match_df = pd.read_csv('/Users/yamamotobikutorueiichi/codes/custom_paris/starwars-swg-gold.csv')


In [163]:
one_to_one_df = pd.read_csv('/Users/yamamotobikutorueiichi/codes/custom_paris/both func and func inv method/sww-swg/multi_filter.csv')


In [164]:
saved_triples_df = pd.read_csv(
    '/Users/yamamotobikutorueiichi/codes/custom_paris/both func and func inv method/sww-swg/saved_triples_9.csv',
    names=['e1', 'p1', 'o1', 'e2', 'p2', 'o2', 'sim'])


/var/folders/6f/k2t20zbs4db5khp6mgdq_06m0000gn/T/ipykernel_44734/584498097.py:1: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  saved_triples_df = pd.read_csv(


In [165]:
pd.set_option('display.max_colwidth', None)


In [166]:
one_to_one_df

,Unnamed: 0,e1,e2,sim
0,0,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Luck,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Luck,0.95000
1,1,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Science_Outpost,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Dathomir_Science_Outpost,0.95000
2,2,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Didina_Lippinoss,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Didina_Lippinoss,0.95000
3,3,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ronin_Lightrunner,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Ronin_Lightrunner,0.95000
4,4,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Fixer_One,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Fixer_One,0.95000
...,...,...,...,...
31420,31420,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Saber%27s_Tooth,http://swg.wikia.com/wiki/File:Nec_cave.jpg,0.02402
31421,31421,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Wasted_Plains,http://swg.wikia.com/wiki/File:Model_3-NIi.jpg,0.02402
31422,31422,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Wasted_Plains,http://swg.wikia.com/wiki/File:Model_3-NIi.jpg,0.02402
31423,31423,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Wasted_Plains,http://swg.wikia.com/wiki/File:Nec_2.jpg,0.02402


In [167]:
match_resource_df = match_df[match_df['e1'].str.contains('/resource/')]
match_set = set(zip(match_resource_df['e1'], match_resource_df['e2']))
one_to_one_df['Predicted'] = False
one_to_one_df['Analyzed'] = False
analyzed_e1_set = set(match_resource_df['e1'])
analyzed_e2_set = set(match_resource_df['e2'])


for index, row in tqdm(one_to_one_df.iterrows(), total=len(one_to_one_df)):
    e1 = row['e1']
    e2 = row['e2']
    
    if (e1, e2) in match_set:
        one_to_one_df.at[index, 'Predicted'] = True
    
    if e1 in analyzed_e1_set or e2 in analyzed_e2_set:
        one_to_one_df.at[index, 'Analyzed'] = True

  0%|          | 0/31425 [00:00<?, ?it/s]

In [168]:
one_to_one_df

,Unnamed: 0,e1,e2,sim,Predicted,Analyzed
0,0,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Luck,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Luck,0.95000,False,False
1,1,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Science_Outpost,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Dathomir_Science_Outpost,0.95000,True,True
2,2,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Didina_Lippinoss,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Didina_Lippinoss,0.95000,True,True
3,3,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ronin_Lightrunner,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Ronin_Lightrunner,0.95000,True,True
4,4,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Fixer_One,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Fixer_One,0.95000,True,True
...,...,...,...,...,...,...
31420,31420,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Saber%27s_Tooth,http://swg.wikia.com/wiki/File:Nec_cave.jpg,0.02402,False,False
31421,31421,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Wasted_Plains,http://swg.wikia.com/wiki/File:Model_3-NIi.jpg,0.02402,False,False
31422,31422,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Wasted_Plains,http://swg.wikia.com/wiki/File:Model_3-NIi.jpg,0.02402,False,False
31423,31423,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Wasted_Plains,http://swg.wikia.com/wiki/File:Nec_2.jpg,0.02402,False,False


In [169]:
analyzed_df = one_to_one_df[one_to_one_df['Analyzed']]

In [170]:
analyzed_df

,Unnamed: 0,e1,e2,sim,Predicted,Analyzed
1,1,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Science_Outpost,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Dathomir_Science_Outpost,0.950000,True,True
2,2,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Didina_Lippinoss,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Didina_Lippinoss,0.950000,True,True
3,3,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ronin_Lightrunner,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Ronin_Lightrunner,0.950000,True,True
4,4,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Fixer_One,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Fixer_One,0.950000,True,True
12,12,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Geonosian_Bio-Lab,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Geonosian_Caves,0.950000,True,True
...,...,...,...,...,...,...
23776,23776,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Gungan_High_Council,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Recover_the_Gungan_Artifact_stolen_by_the_Muskegs,0.692134,False,True
24310,24310,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Wyyyschokk,http://dbkwik.webdatacommons.org/swg.wikia.com/property/meat,0.656538,False,True
24661,24661,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Sinis,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Crash_and_Burn,0.686888,False,True
25465,25465,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Zsuzsanna_Cseh,"http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Forward_Watch,_Gamma_Camp",0.648675,False,True


In [171]:
false_positive_df = analyzed_df[analyzed_df['Predicted'] == False]

In [172]:
false_positive_df[false_positive_df['sim'] > 0.9][:-20]

,Unnamed: 0,e1,e2,sim,Predicted,Analyzed
25,25,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/KSE_Firespray,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/KSE_Firespray,0.95,False,True
37,37,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/IG-227_Hailfire-class_droid_tank,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Hailfire_Droid,0.95,False,True
61,61,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Cybernetics,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Cybernetic,0.95,False,True
75,75,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Life_Day,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Life_Day,0.95,False,True
82,82,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Doombringer,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Doom_Bringer,0.95,False,True
153,153,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ye-4_gunship,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Ye-4_Gunship,0.95,False,True
156,156,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Maximilian_Veers,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Maximilian_Veers,0.95,False,True
290,290,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Durindfire,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Durindfire,0.95,False,True
291,291,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Flameout,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Flameout,0.95,False,True
325,325,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/YT-2400_light_freighter,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/YT-2400,0.95,False,True


In [203]:
mask_1 = saved_triples_df['e1'] == 'http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ezkiel'
mask_2 = saved_triples_df['e2'] == 'http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Brother_Ezkiel'
mask = mask_1 & mask_2


In [204]:
for index, row in saved_triples_df[mask].iterrows():
    if row['sim'] > 0.5:
        print(row['e1'])
        print(row['p1'])
        print(row['o1'])
        print(row['e2'])
        print(row['p2'])
        print(row['o2'])
        print(row['sim'])
        print('_____')

http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ezkiel
http://dbkwik.webdatacommons.org/ontology/thumbnail
http://starwars.wikia.com/wiki/Special:FilePath/Brotherezkiel.jpg?width=300
http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Brother_Ezkiel
http://dbkwik.webdatacommons.org/ontology/thumbnail
http://swg.wikia.com/wiki/Special:FilePath/Brotherezkiel.jpg?width=300
0.5443308771928342
_____
http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ezkiel
http://dbkwik.webdatacommons.org/ontology/abstract
Brother Ezkiel was a male Ishi Tib who served the Rebel Alliance's Vortex Squadron on the planet Dathomir during the Galactic Civil War.



http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Brother_Ezkiel
http://dbkwik.webdatacommons.org/ontology/abstract
Located in the Abandoned Rebel Base on Dantooine, Brother Ezkiel is the trainer and mission giver for Tier 3 Alliance Starfighter Pilots of the Vortex Squadron. /way dantooine -6824 5547 Brother Ezkiel
To

In [185]:
paris_one_to_one_df = pd.read_csv('/Users/yamamotobikutorueiichi/codes/paris/sww-swg/one_to_one.csv')

In [186]:
paris_one_to_one_df

,Unnamed: 0,e1,e2,sim
0,0,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Talon_Karrde,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Talon_Karrde,1.000000
1,1,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Theed,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Theed,1.000000
2,2,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Luke_Skywalker,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Luke_Skywalker,1.000000
3,3,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Alliance_to_Restore_the_Republic,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Rebel_Alliance,1.000000
4,4,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Reelo_Baruk,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Reelo_Baruk,1.000000
...,...,...,...,...
3646,3646,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Tango_Company,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Misc,0.100685
3647,3647,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/The_Galactic_Empire—Ships_of_the_Fleet,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/TIE_Fighter_Chassis_Blueprints_(Schematic),0.100671
3648,3648,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/C-14A_stun_grenade,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Commando_Fundamentals_II,0.100334
3649,3649,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Torani_Kulda,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Blix%27s_Blaster,0.100054


In [187]:
match_resource_df = match_df[match_df['e1'].str.contains('/resource/')]
match_set = set(zip(match_resource_df['e1'], match_resource_df['e2']))
paris_one_to_one_df['Predicted'] = False
paris_one_to_one_df['Analyzed'] = False
analyzed_e1_set = set(match_resource_df['e1'])
analyzed_e2_set = set(match_resource_df['e2'])


for index, row in tqdm(paris_one_to_one_df.iterrows(), total=len(paris_one_to_one_df)):
    e1 = row['e1']
    e2 = row['e2']
    
    if (e1, e2) in match_set:
        paris_one_to_one_df.at[index, 'Predicted'] = True
    
    if e1 in analyzed_e1_set or e2 in analyzed_e2_set:
        paris_one_to_one_df.at[index, 'Analyzed'] = True

  0%|          | 0/3651 [00:00<?, ?it/s]

In [188]:
paris_one_to_one_df

,Unnamed: 0,e1,e2,sim,Predicted,Analyzed
0,0,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Talon_Karrde,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Talon_Karrde,1.000000,True,True
1,1,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Theed,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Theed,1.000000,False,False
2,2,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Luke_Skywalker,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Luke_Skywalker,1.000000,False,False
3,3,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Alliance_to_Restore_the_Republic,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Rebel_Alliance,1.000000,False,False
4,4,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Reelo_Baruk,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Reelo_Baruk,1.000000,True,True
...,...,...,...,...,...,...
3646,3646,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Tango_Company,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Misc,0.100685,False,False
3647,3647,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/The_Galactic_Empire—Ships_of_the_Fleet,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/TIE_Fighter_Chassis_Blueprints_(Schematic),0.100671,False,False
3648,3648,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/C-14A_stun_grenade,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Commando_Fundamentals_II,0.100334,False,False
3649,3649,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Torani_Kulda,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Blix%27s_Blaster,0.100054,False,False


In [189]:
true_paris_df = paris_one_to_one_df[paris_one_to_one_df['Predicted'] == True]

In [190]:
key_cols = ['e1', 'e2']

analyzed_df['key'] = analyzed_df[key_cols].apply(lambda x: '_'.join(x.astype(str)), axis=1)
true_paris_df['key'] = true_paris_df[key_cols].apply(lambda x: '_'.join(x.astype(str)), axis=1)

# Using isin() and boolean indexing
result_df = true_paris_df[~true_paris_df['key'].isin(analyzed_df['key'])]

/var/folders/6f/k2t20zbs4db5khp6mgdq_06m0000gn/T/ipykernel_44734/3613503917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analyzed_df['key'] = analyzed_df[key_cols].apply(lambda x: '_'.join(x.astype(str)), axis=1)
/var/folders/6f/k2t20zbs4db5khp6mgdq_06m0000gn/T/ipykernel_44734/3613503917.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_paris_df['key'] = true_paris_df[key_cols].apply(lambda x: '_'.join(x.astype(str)), axis=1)


In [191]:
result_df

,Unnamed: 0,e1,e2,sim,Predicted,Analyzed,key
94,94,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Pooja_Naberrie/Legends,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Pooja_Naberrie,1.000000,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Pooja_Naberrie/Legends_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Pooja_Naberrie
1310,1310,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Trade_Outpost,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Dathomir_Trade_Outpost,0.999999,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Trade_Outpost_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Dathomir_Trade_Outpost
1463,1463,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/FC-20_speeder_bike/Legends,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Sith_Speeder,0.999960,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/FC-20_speeder_bike/Legends_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Sith_Speeder
1528,1528,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Massiff/Legends,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Massiff_House_Pet,0.999894,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Massiff/Legends_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Massiff_House_Pet
1529,1529,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Lava_flea/Legends,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Lava_Flea,0.999891,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Lava_flea/Legends_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Lava_Flea
1713,1713,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Research_Outpost,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Endor_Research_Outpost,0.999813,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Research_Outpost_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Endor_Research_Outpost
1878,1878,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Radiation_zone_assault_trooper,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Radtrooper_Insignia,0.998760,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Radiation_zone_assault_trooper_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Radtrooper_Insignia
1899,1899,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Junk_dealer/Legends,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Junk_Dealer,0.998616,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Junk_dealer/Legends_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Junk_Dealer
2061,2061,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Dera_Darklighter,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Dera,0.993033,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Dera_Darklighter_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Dera
2103,2103,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Pulse_cannon/Legends,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Pulse_Cannon,0.990750,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Pulse_cannon/Legends_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Pulse_Cannon


In [192]:
result_df[-20:]

,Unnamed: 0,e1,e2,sim,Predicted,Analyzed,key
94,94,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Pooja_Naberrie/Legends,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Pooja_Naberrie,1.000000,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Pooja_Naberrie/Legends_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Pooja_Naberrie
1310,1310,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Trade_Outpost,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Dathomir_Trade_Outpost,0.999999,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Trade_Outpost_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Dathomir_Trade_Outpost
1463,1463,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/FC-20_speeder_bike/Legends,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Sith_Speeder,0.999960,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/FC-20_speeder_bike/Legends_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Sith_Speeder
1528,1528,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Massiff/Legends,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Massiff_House_Pet,0.999894,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Massiff/Legends_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Massiff_House_Pet
1529,1529,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Lava_flea/Legends,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Lava_Flea,0.999891,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Lava_flea/Legends_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Lava_Flea
1713,1713,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Research_Outpost,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Endor_Research_Outpost,0.999813,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Research_Outpost_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Endor_Research_Outpost
1878,1878,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Radiation_zone_assault_trooper,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Radtrooper_Insignia,0.998760,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Radiation_zone_assault_trooper_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Radtrooper_Insignia
1899,1899,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Junk_dealer/Legends,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Junk_Dealer,0.998616,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Junk_dealer/Legends_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Junk_Dealer
2061,2061,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Dera_Darklighter,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Dera,0.993033,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Dera_Darklighter_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Dera
2103,2103,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Pulse_cannon/Legends,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Pulse_Cannon,0.990750,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Pulse_cannon/Legends_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Pulse_Cannon


In [193]:
key_cols = ['e1', 'e2']

predicted_df = analyzed_df[analyzed_df['Predicted']]
predicted_df['key'] = predicted_df[key_cols].apply(lambda x: '_'.join(x.astype(str)), axis=1)
analyzed_paris_df = paris_one_to_one_df[paris_one_to_one_df['Analyzed']]
analyzed_paris_df['key'] = analyzed_paris_df[key_cols].apply(lambda x: '_'.join(x.astype(str)), axis=1)

# Using isin() and boolean indexing
result_df = predicted_df[~predicted_df['key'].isin(analyzed_paris_df['key'])]

/var/folders/6f/k2t20zbs4db5khp6mgdq_06m0000gn/T/ipykernel_44734/870593297.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_df['key'] = predicted_df[key_cols].apply(lambda x: '_'.join(x.astype(str)), axis=1)
/var/folders/6f/k2t20zbs4db5khp6mgdq_06m0000gn/T/ipykernel_44734/870593297.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analyzed_paris_df['key'] = analyzed_paris_df[key_cols].apply(lambda x: '_'.join(x.astype(str)), axis=1)


In [202]:
result_df[-20:]

,Unnamed: 0,e1,e2,sim,Predicted,Analyzed,key
3114,3114,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ezkiel,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Brother_Ezkiel,0.850561,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ezkiel_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Brother_Ezkiel
3318,3318,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Rikkh,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Captain_Rikkh,0.843276,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Rikkh_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Captain_Rikkh
3328,3328,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Lok_Marathon,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Lok_Marathon_Race,0.842823,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Lok_Marathon_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Lok_Marathon_Race
3444,3444,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Bloodguard,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Death_Watch_Bloodguard,0.839879,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Bloodguard_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Death_Watch_Bloodguard
3615,3615,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Narl,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Colonel_Narl,0.836149,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Narl_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Colonel_Narl
3669,3669,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/M3-A_Scyk_fighter,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/MandalMotors_M3-A_%22Scyk%22_Interceptor,0.835258,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/M3-A_Scyk_fighter_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/MandalMotors_M3-A_%22Scyk%22_Interceptor
3689,3689,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Luilris_mushroom_pie,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Luilris_Mushrooms,0.834712,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Luilris_mushroom_pie_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Luilris_Mushrooms
3705,3705,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ramna,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Commander_Ramna,0.834405,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ramna_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Commander_Ramna
3799,3799,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ufwol,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/General_Ufwol,0.832384,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Ufwol_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/General_Ufwol
4175,4175,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Jinkins,http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Find_Jinkins,0.826682,True,True,http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Jinkins_http://dbkwik.webdatacommons.org/swg.wikia.com/resource/Find_Jinkins


In [199]:
paris_one_to_one_df[paris_one_to_one_df['e1'] == 
                    'http://dbkwik.webdatacommons.org/starwars.wikia.com/resource/Jedi']

,Unnamed: 0,e1,e2,sim,Predicted,Analyzed
